In [1]:
#divergencias
import util
from backtesting import Backtest
import estrategias as est

lista = ['ETHUSDT','BCHUSDT','XRPUSDT','ADAUSDT','FILUSDT','MATICUSDT','OPUSDT','FETUSDT','BTCUSDT'] # lista acotada

#lista=['XRPUSDT']

timeframe = '1h'

###
razon_max = 0.0
razon = 0.0
suma = 0.0
trades = 0
for largo in [1]:
    try:
        suma = 0.0
        trades = 0
        razon = 0.0
        for symbol in lista:
            try:    
                #data = est.estrategia_lucho (symbol,timeframe)
                data = est.estrategia_adx (symbol,timeframe)
                bt = Backtest(data,util.backtesting_config,cash=1000000,exclusive_orders=True)
                output = bt.run()
                if len(lista)==1:
                    bt.plot(filename="graficos/"+symbol)
                print(f"Symbol: {symbol} - Return [%]: {round(output['Return [%]'],2)} - # Trades: {output['# Trades']} - Win Rate [%]: {output['Win Rate [%]']}")
                suma=suma+output['Return [%]']
                trades=trades+output['# Trades']
            except Exception as e:
                print(f"Error en {symbol}: {e}")
        razon = suma/trades
        if razon > razon_max:
            razon_max = razon
        print(f"Suma total: {suma} - Trades total: {trades}")
        print(f"razon_max: {razon_max}")
    except Exception as e:
        print(f"Error en {symbol}: {e}")
        print(f"Suma total: {suma} - Trades total: {trades}")
        print(f"razon_max: {razon_max}")
print(f"razon_max: {razon_max}")

Symbol: ETHUSDT - Return [%]: 19.02 - # Trades: 9 - Win Rate [%]: 77.77777777777779
Symbol: BCHUSDT - Return [%]: -3.05 - # Trades: 8 - Win Rate [%]: 12.5
Symbol: XRPUSDT - Return [%]: -5.96 - # Trades: 11 - Win Rate [%]: 9.090909090909092
Symbol: ADAUSDT - Return [%]: 7.99 - # Trades: 9 - Win Rate [%]: 44.44444444444444
Symbol: FILUSDT - Return [%]: 8.08 - # Trades: 7 - Win Rate [%]: 57.14285714285714
Symbol: MATICUSDT - Return [%]: 10.6 - # Trades: 6 - Win Rate [%]: 66.66666666666666
Symbol: OPUSDT - Return [%]: 10.08 - # Trades: 11 - Win Rate [%]: 36.36363636363637
Symbol: FETUSDT - Return [%]: 3.8 - # Trades: 11 - Win Rate [%]: 27.27272727272727
Symbol: BTCUSDT - Return [%]: 12.91 - # Trades: 8 - Win Rate [%]: 62.5
Suma total: 63.479551981207074 - Trades total: 80
razon_max: 0.7934943997650884
razon_max: 0.7934943997650884
